In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # es para enseñar todas las columnas del df

import warnings
warnings.simplefilter('ignore')   # es para quitar warnings

In [2]:
from selenium import webdriver

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


In [4]:
import requests as req

### Web scraping

In [5]:
PATH=ChromeDriverManager().install()
driver=webdriver.Chrome(PATH)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\usuario\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [6]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'

driver.get(url)

In [8]:
# Scrapeo de una tabla

tbody=driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/table[5]/tbody')

filas=tbody.find_elements_by_tag_name('tr')


data=[[e.text for e in f.find_elements_by_tag_name('td')] for f in filas]

cabeceras=driver.find_element_by_tag_name('thead')

cabeceras=[c.text for c in cabeceras.find_elements_by_tag_name('th')][1:]

In [9]:
cities = pd.DataFrame(data)

cities = cities.drop(cities.columns[[5, 7]], axis=1)

In [10]:
cities.columns = cabeceras

In [11]:
driver.quit()

### API request (Balldontlie)

In [12]:
import time

In [13]:
# balldontlie genérica para jugadores


URL_BASE='https://www.balldontlie.io/api/v1/players?per_page=100&page='

In [14]:
# expecifica por id de jugador

# player_id = '237'

In [15]:
# Filtros

# page=1  nº de página

# perpage=100 # Max 100 per page


In [16]:
# Hago un bucle para recorrer las 38 páginas de 100 jugadores y hago un concat en la lista jugadores
# Le pongo 1 seg de sleep porque la api tiene límite por minuto

players_data=[]



for i in range(1, 39):

    
    res=req.get(URL_BASE+str(i))
    
    players_data += res.json()['data']
    
    time.sleep(1)

In [17]:
player_dim = pd.DataFrame(players_data)

In [18]:
player_dim[player_dim.id == 237]

,id,first_name,height_feet,height_inches,last_name,position,team,weight_pounds
3565,237,LeBron,6.0,8.0,James,F,"{'id': 14, 'abbreviation': 'LAL', 'city': 'Los...",250.0


### Export to csv

In [20]:
cities.to_csv('../data/data_scraping/cities.csv', sep= ",", index=False)
player_dim.to_csv('../data/data_scraping/player_dim.csv', sep= ",", index=False)